# Tutorial: Deep Learning for PDEs - Comparing FEM, PINN, and GNN

Welcome to this introductory tutorial on solving Partial Differential Equations (PDEs) using neural networks. This notebook is designed to guide you through the transition from traditional numerical methods (FEM) to modern deep learning approaches (PINNs and GNNs).

## Introduction: The Poisson Equation

The **Poisson equation** is a fundamental elliptic PDE used in physics and engineering to describe electrostatics, steady-state heat conduction, and fluid flow.

### Mathematical Formulation
We seek to find a scalar function $u(x, y)$ that satisfies:

$$-\Delta u(x, y) = f(x, y) \quad \forall (x, y) \in \Omega$$
$$u(x, y) = 0 \quad \forall (x, y) \in \partial\Omega$$

Where:
- $\Omega$: The **spatial domain**, defined here as a unit square $[0, 1] \times [0, 1]$.
- **Time Domain**: Since this is a steady-state problem, the **simulation time** is $t=0$. The solution represents a static equilibrium. However, the models are built to anticipate future time-dependent variables $u(x, t)$.
- $\Delta$: The Laplace operator $\nabla \cdot \nabla$, representing diffusion or curvature.
- $f(x, y)$: The **source term**, which "drives" the solution (e.g., heat generation or charge density).

### Modeling Approaches
1. **Finite Element Method (FEM)**: A classical mesh-based technique that discretizes the space into simple elements (triangles).
2. **Physics-Informed Neural Networks (PINN)**: A continuous, mesh-free approach that solves the PDE by minimizing residuals using Autograd.
3. **Graph Neural Networks (GNN)**: A physics-aware modern approach that operates on the mesh graph, learning to map topology to solutions.


In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader
import pytorch_lightning as pl

# Local modules
from FEM.fem_solver import get_problem
from config.physics import *
from utils.plotting import *
from utils.metrics import *
from utils.geometry import *
from utils.train_utils import * 
from utils.reporting import *

pl.seed_everything(42)

import torch
if torch.cuda.is_available():
    # Inicializa el contexto de CUDA explícitamente
    torch.cuda.set_device(0)
    torch.zeros(1).cuda()
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

## 1. Defining the Domain: The Geometry Module

Before solving any physics equation, we must define the **Domain** ($\Omega$) where the phenomenon occurs. In this tutorial, we move away from "hardcoded" coordinates to a modular approach using `utils/geometry.py`.

The **Geometry Object** acts as the single source of truth for our entire pipeline:
* **Limits**: Defines the physical boundaries (e.g., a square from 0 to 1).
* **Sampling**: It knows how to generate points inside the domain ($\Omega$) and on its boundaries ($\partial\Omega$).

By centralizing the geometry, if you decide to change the size of the plate or even its shape, you only need to modify the geometry initialization. This ensures that when we compare **FEM**, **PINN**, and **GNN**, we are all looking at the exact same "piece of reality".

* **For FEM**: We will discretize this geometry into a **Mesh** (nodes and elements).
* **For PINN**: We will treat it as a **Point Cloud** (sampling points inside and on the boundaries).
* **For GNN**: We will treat it as a **Graph** (nodes as vertices, connections as edges).

In [ ]:
# Define the physical limits of our problem
# This object will be the 'source of truth' for all methods
geom = SquareGeometry(x_range=[0, 1], y_range=[0, 1], device=device)

print(f"Geometry initialized: Square domain from {geom.x_range} to {geom.y_range}")

## 2. The Numerical Baseline: Finite Element Method (FEM)

To evaluate if a Neural Network is "smart," we first need a reliable reference. The **Finite Element Method (FEM)** is the industry standard for solving PDEs like the Poisson equation.

Unlike neural networks (PINNs), which are continuous, FEM is a **discrete method**. It transforms our continuous geometry into a collection of small, simple shapes (elements). 

### From Geometry to Mesh
The FEM cannot work with raw ranges; it needs a **Mesh**. We take our `geom` object and discretize it.

* **Nodes**: Specific coordinates where the solution $u$ is calculated.
* **Elements**: Triangles or quads that connect the nodes.

We use the `skfem` library to:
1.  **Discretize** the domain.
2.  **Assemble** the physics (Laplacian and Source terms).
3.  **Solve** the resulting linear system to get the "Ground Truth" solution.

This solution will be our gold standard to measure the accuracy of our PINN and GNN models later.

In [ ]:
# ==========================================
# STEP 1: DEFINE PHYSICS & MESH PARAMETERS
# ==========================================
# You can change 'sine' to 'const' to see how the solution changes

nelem = 2               # Mesh resolution (nelem x nelem)
porder = 2              # Polynomial order (Lin or cuad elements)
mesh_type = 'quad'       # Mesh type: 'tri' or 'quad'
source_type = 'sine'    # Source term type: 'constant' or 'sine'

prob = get_problem(geometry=geom, nx=nelem, ny=nelem, porder=porder, source_type=source_type, mesh_type=mesh_type)
u_exact = prob['u_exact']
doflocs = prob['basis'].doflocs.T

print(f"Configuring FEM for {source_type} source term...")
phys = PoissonPhysics(source_type=source_type)
print(f"FEM Solution ready with {len(doflocs)} Degree of Freedom (DOF).")

### 2.1 Mesh Configuration: From Continuous to Discrete

Creating a mesh is the process of defining the resolution of our "numerical microscope". In this step, we configure how the geometry will be divided. 

The main parameters are:
* **`nelem` (Resolution)**: The number of divisions along each axis. A higher number captures more detail but requires more memory.
* **`porder` (Polynomial Order)**: Defines the complexity of the solution inside each element. 
    * *Order 1 (Linear)*: The solution is a straight line between nodes.
    * *Order 2 (Quadratic)*: The solution can curve within the element, offering much higher precision.
* **`mesh_type`**: We can use triangles (`tri`) for complex shapes or quadrilaterals (`quad`) for regular grids.

We use a structured triangular mesh with the following parameters (aligned with the `CircleDeterministic.py` reference):

| Parameter | Description | Default Value |
| :--- | :--- | :--- |
| `nelem` | Number of subdivisions along one side | 2 |
| `porder` | Polynomial order of the basis functions | 2 (Quadratic) |
| `source` | Source term type $f(x, y)$ | 'sine' |
| `domain` | Spatial extent | $[0, 1]^2$ |
| `t_sim`  | Simulation Time | Static ($t=0$) |

In [ ]:
# Create a dictionary from your notebook variables for the report
FEM_CONFIG = {
    'nelem': nelem,
    'porder': porder,
    'mesh_type': mesh_type,
    'source_type': source_type
}

# Display summary
print_config_summary(model=prob, config=FEM_CONFIG, model_type="FEM")

### 2.2 Visualization: The Calculation Skeleton

Visualization is key to ensuring our discretization is correct. Below, we plot the generated mesh. 

In this plot, you are seeing the **Topology** of the problem:
* **Nodes (Vertices)**: The specific points where the FEM calculates the value of $u(x, y)$. In GNN terms, these will be our **Graph Nodes**.
* **Edges/Elements**: The connections between nodes. In GNN terms, these define the **Graph Edges**.

By passing our `geom` object to the `get_problem` function, the mesh is automatically scaled to the correct physical dimensions, ensuring that our "skeleton" perfectly matches the real-world domain we defined earlier.

In [ ]:
# ==========================================
# STEP 2: AUTOMATIC MESH VISUALIZATION
# ==========================================
# 1. Mesh configuration based on user parameters
prob = get_problem(
    geometry=geom,  
    nx=nelem,
    ny=nelem, 
    porder=porder, 
    mesh_type=mesh_type, 
    source_type=source_type,
)

# 2. Automatic mesh plotting function tri/cuad
plot_fem_mesh(
    prob, 
    title=f"Mesh: {mesh_type.upper()} | Order: P{porder} | Res: {nelem}x{nelem}"
)

plt.show()

# =========================================
print(f"--- Ficha Técnica de la Malla ---")
print(f"Estructura: {mesh_type.upper()} | Orden: P{porder}")
print(f"Resolución: {nelem}x{nelem} elementos")
print(f"Total DoFs (Nodos de cálculo): {len(prob['doflocs'])}")

### 2.3 FEM Results: Establishing the Ground Truth

After solving the system, we obtain the distribution of $u(x, y)$—for example, the steady-state temperature across the plate. 

This result is our **Reference Solution**. We will use it for:
1.  **Direct Comparison**: To visually check if the PINN looks similar.
2.  **Error Metrics**: To calculate the exact L2 error of the Neural Network.
3.  **GNN Training**: In supervised learning, this FEM data will be the "label" or "target" that the Graph Neural Network will try to mimic.

Pay attention to the color map: it represents the physical response of the system to the source term $f(x, y)$ we defined.

In [ ]:
# ==========================================
# STEP 3: SOLVE FEM and VISUALIZE GROUND TRUTH
# ==========================================

# Validation plots
plot_fem_validation(prob, title=f"Validation: {mesh_type} P{porder} (Res: {nelem})")
plt.show()

# Error calculation
l2_error = np.sqrt(np.mean((prob['u'] - prob['u_exact'])**2))
print(f"Mean Squared Error (Nodes): {l2_error:.6f}")

df_metrics, raw_metrics = calculate_fem_metrics(prob['u'], prob['u_exact'])
print(f"REPORT: {mesh_type} P{porder}")
display(df_metrics)


## 3. Physics-Informed Neural Networks (PINNs)

The core idea of a PINN is to use a Neural Network as a **universal function approximator** $u_{\theta}(x, y)$ that satisfies a differential equation. 

Unlike the FEM approach we just saw, PINNs are **mesh-free**. Instead of dividing the space into elements, we evaluate the "physical health" of the network at random points. If the network violates the laws of physics (the PDE), we penalize it through the Loss Function.

The Neural Network ($u_\theta$)
Usually a simple **Multilayer Perceptron (MLP)**. It takes spatial coordinates $(x, y)$ as input and outputs the physical quantity of interest $u$.
* **Input:** Coordinates $(x, y)$
* **Output:** Predicted solution $\hat{u}(x, y)$

To guarantee a good solution, a PINN must balance two different goals:
1.  **Physics Loss ($\mathcal{L}_{pde}$)**: Does the network satisfy $-\Delta u = f$ inside the domain?
2.  **Boundary Loss ($\mathcal{L}_{bc}$)**: Does the network respect the fixed values (e.g., $u=0$) at the edges?

### 3.1 The Engine: Automatic Differentiation

The "secret sauce" that allows PINNs to solve PDEs is **Automatic Differentiation (AD)**. 

In traditional methods, we approximate derivatives using finite differences (slopes between mesh nodes). In PINNs, we use the same mechanism that trains standard AI (backpropagation) to calculate **exact analytical derivatives** of the network's output with respect to its input coordinates $(x, y)$.

$$\frac{\partial u}{\partial x} \approx \text{torch.autograd.grad}(u, x)$$

This means our "sensor" for physics is continuous: we can calculate the Laplacian $\Delta u$ at **any** coordinate, not just on a predefined grid.

### 3.2 The Physics-Informed Loss Function

Training a PINN is an optimization problem. We define a total loss function that guides the network toward the correct physical behavior:


$$\mathcal{L}_{total} = \omega_{pde} \mathcal{L}_{pde} + \omega_{bc} \mathcal{L}_{bc}$$

* **Residual Loss ($\mathcal{L}_{pde}$)**: We sample $N_{col}$ points inside the domain and minimize the residual. For the Poisson equation $-\Delta u = f$ the residual is $r := -\Delta u - f$. If $r=0$, the physics are perfectly satisfied. Thus, the loss function is defined to minimize the residual:
  $$\mathcal{L}_{PDE} = \frac{1}{N_{col}} \sum_{i=1}^{N_{col}} |-\Delta u_\theta(x_i) - f(x_i)|^2$$

* **Boundary Loss ($\mathcal{L}_{bc}$)**: We sample $N_{bc}$ points on the edges and force the network to match the boundary conditions (Dirichlet, Neumann, etc.): 
    $$\mathcal{L}_{BC} = \frac{1}{N_{bc}} \sum_{j=1}^{N_{bc}} |u_\theta(x_j) - 0|^2$$


> **Student Note**: The weights $\omega$ (like `lambda_bc` in our code) are crucial. Often, the boundary conditions are harder to learn than the interior physics, so we give them more "importance" during training.

### 3.3 Collocation Points: Where the Physics Happens

These are the points $x_i$ sampled randomly within the domain $\Omega$ where we evaluate the PDE residual. Since the PINN is **mesh-free**, we can sample these points dynamically at every iteration or use a fixed set of points to guide the learning of the underlying physics.

In this notebook, you can choose between two strategies for the points where the physics are evaluated:
1.  **Random Collocation**: High flexibility. The network explores the domain freely.
2.  **FEM-based Nodes**: We use the exact same nodes as the FEM mesh. This is useful for direct 1-to-1 comparisons and to see if the network can "match" the classical solution at specific points.

**Why use random points?** Random sampling (especially if it's dynamic) prevents the network from "memorizing" the solution at specific locations, forcing it to learn the global underlying physical law.

### 3.4 Setting up the PINN Experiment

#### Key Hyperparameters to watch:
* **Max Epochs**: How many times the network will look at the domain.
* **Learning Rate**: The size of the step the optimizer takes. If it's too high, the physics might "explode"; if it's too low, the network will never learn the PDE.
* **Accelerator**: We use `auto` to detect if you have a GPU (CUDA) available, which significantly speeds up the Autograd process.

#### Train, test & validation datasets.

In a PINN, we don't have a "fixed dataset" of images or labels; we have an infinite number of coordinates $(x, y)$ in our domain. You might ask: If the physics are universal, shouldn't I use all possible points for training to get the perfect solution? The answer is No, and here is why: 

1. The Overfitting Trap in Physics
A Neural Network is an incredibly flexible optimizer. If we only evaluate the PDE residual on a fixed, static set of training points, the network might find a "mathematical shortcut": it makes the residual zero exactly at those points while creating wild, non-physical oscillations (instabilities) in between them.

Validation acts as an independent judge. It checks coordinates the network has never "seen" to ensure the solution is smooth and physically consistent everywhere. If the training error is low but the validation error is high, your network is "memorizing" the points instead of "understanding" the PDE.

2. Stability and Convergence
Physical systems are sensitive to boundaries and gradients. By monitoring a separate Validation Set, we can detect if the solution is becoming unstable or "exploding" before the training ends. It is our primary tool to ensure the numerical stability of the neural solver.

In a standard AI project, you divide your "data" into three sets. In a PINN, we do the same, but instead of "data samples", we divide our **Geometry**:

-  **Training Set (Collocation Points)**: The training set is the portion of the data used to fit the model parameters, allowing the algorithm to learn patterns from examples. In PINNs, these are the coordinates $(x, y)$ where the neural network will evaluate the PDE. The "loss" here is the physics residual. 

-  **Validation Set**: The validation set is used during development to tune hyperparameters, evaluate convergence and monitor overfitting. 

-  **Test Set (Ground Truth Comparison)**: If available, it measures real accuracy of the model. It is an independent and unbiased evaluation of the final model’s predictive performance.

**Why is this relevant?** Because it allows us to prove that the PINN can generalize the solution to the entire domain, not just the points it "saw" during training.

#### Data loaders 

The **DataLoader** is the pipe that connects our points to the GPU. 

* **Format**: In PINNs, the DataLoader delivers a dictionary or a tensor of **raw coordinates** $(x, y)$. 
* **Batches**: We don't feed all points at once. We send them in small groups (batches). For example, if we have 2000 points and a `batch_size=32`, the optimizer will update the physics 63 times per epoch.
* **Format in this project**: Our `PinnDataset` organizes two types of inputs: `'pde'` (interior points) and `'bc'` (boundary points).

## 3.6. STEP 1: Initializing the PINN Strategy

Now we move to the code. In this first step, we define the "Brain" (Hyperparameters) and the "Eyes" (Collocation points).

**CRITICAL CONCEPT**: Notice that in the following cell, we are **NOT** loading any experimental data or solutions. We are only generating **coordinates**. 
* We define how many points we want in the interior (`n_train`).
* We define how many points we want on the boundaries (`n_bc`).
* We visualize them to ensure they cover the domain $\Omega$ correctly.

The PINN starts completely "blind". It only has a set of coordinates and a mathematical rule (the PDE) to follow.

In [ ]:
# ============================================================
# STEP 1: DATA GENERATION & STRATEGY
# ============================================================
import torch
import numpy as np
import matplotlib.pyplot as plt
from PINN.pinn_module import PinnDataset, ValDataset

# --- Manual Control Dashboard ---
use_fem_for_train = False  
use_fem_for_test  = False   
n_train, n_test, n_bc = 100, 100, 100
lambda_bc = 10.0
# --- Learning config ---
hidden_dim = 50
n_layers = 4
lr = 1e-3
max_epochs = 500
batch_size = 25
# --- Physics config ---
source_type = 'sine'  # 'const' or 'sine'  
my_physics = PoissonPhysics(source_type=source_type)

# 1. Collocation points (Interior)
if use_fem_for_train:
    train_pts = torch.tensor(prob['doflocs'], dtype=torch.float32)
    train_label = "FEM Nodes"
else:
    train_pts = geom.sample_interior(n_train)
    train_label = "Random Collocation"

# 2. Boundary points (BCs) - 2D Logics
bc_coords = geom.sample_boundary(n_bc)
side_mask = torch.randint(0, 4, (n_bc,))
bc_coords[side_mask==0, 0] = 0.0; bc_coords[side_mask==1, 0] = 1.0
bc_coords[side_mask==2, 1] = 0.0; bc_coords[side_mask==3, 1] = 1.0

# 3. Test/Validation points
if use_fem_for_test:
    test_pts = torch.tensor(prob['doflocs'], dtype=torch.float32)
    test_val = torch.tensor(prob['u'], dtype=torch.float32).view(-1, 1)
    test_label = "FEM Mesh"
else:
    test_pts = geom.sample_interior(n_test)
    test_val = None             # Random points
    test_label = "Random Test"

# 4. DataLoaders en PinnModule
train_ds = PinnDataset(geometry=geom, pde_pts=train_pts, bc_pts=bc_coords)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)

val_ds = ValDataset(geometry=geom, pts=test_pts, vals=test_val)
val_loader = DataLoader(val_ds, batch_size=len(test_pts))

# --- Visualización Adaptativa ---
plot_pinn_strategy(
    train_pts.detach().cpu().numpy(), 
    bc_coords.detach().cpu().numpy(), 
    test_pts.detach().cpu().numpy(), 
    title=f"Strategy: {train_label} vs {test_label}"
)

### 3.7. STEP 2: Understanding the Physics-Informed Neural Network (PINN)

To solve a PDE like the Poisson equation without a mesh, we use a Neural Network as a function approximator $u_{\theta}(x, y)$. Unlike standard deep learning, we don't only minimize the error against data; we minimize the **residual of the PDE**.

### The Core Components
The project is divided into three main modules that you should understand:
1. **`pinn_config.py` (Predefined configurations)**: Before building the model, we define a dictionary (or a configuration object) called pinn_config. Think of this as the control panel of the neural network.
    - Architecture: Here we define how many neurons and layers the network has.
    - Hyperparameters: We set the learning rate, the weight of the boundary conditions (lambda_bc), and the number of epochs.
    - Physics Link: This is where we tell the PINN which physical law it must follow.

2.  **`physics.py` (The Rules)**: The Mathematical Recipe. The PINN does not "know" physics by default. We must teach it the rules through a specific script: physics.py. This module is designed to be pluggable: you can swap the Poisson equation for any other differential equation (like Fourier or Cattaneo) as long as you provide the following three components:
    - The Source Term ($f$). In our Poisson example, this is the heat source or charge distribution.
       * Poisson case: $f(x, y) = 2\pi^2 \sin(\pi x) \sin(\pi y)$.
       * Implementation: The model evaluates this function at every coordinate $(x, y)$ to know what "force" is acting on the system.
    - The Laplacian and the Residual ($\mathcal{R}$). This is the heart of the PINN. We use Automatic Differentiation to calculate the Laplacian ($\Delta u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}$).
        * The Residual: We define $\mathcal{R} = \Delta u + f$
        * The Goal: If the network is doing its job perfectly, $\mathcal{R}$ should be zero everywhere. The training process is simply the network trying to minimize this residual.
    - Boundary Conditions (BCs). Finally, we define the "rules" at the edges of our domain.
        * Poisson case (Dirichlet): We force $u(x, y) = 0$ at the four walls of the square.
        * Implementation: The physics module checks the network's prediction at the boundary coordinates and calculates the error against the target value (zero).

3.  **`pinn_module.py` (The Engine)**: This contains the `PINNSystem` (built with PyTorch Lightning). It handles the training loops, the optimization (Adam/LBFGS), and the logging of metrics. This has been designed to be agnostic to the underlying physics and dimensionality, so no modifications should be required (in principle). At each training step it:
    1. Takes a batch of coordinates from the DataLoader.
    2. Asks the network for a prediction $u$.
    3. Calls physics.py to calculate how much that prediction violates the PDE.
    4. Computes the Total Loss (PDE + Boundary) and tells the optimizer how to adjust the neurons.

---

> **NOTE** The beauty of this modular structure is that the pinn_module.py (the engine) never changes. To solve a more complex problem, you only update physics.py:


In [ ]:
# ============================================================
# STEP 2: MODEL & TRAINER SETUP
# ============================================================
import torch
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from PINN.pinn_module import PINNSystem
from config.physics import *
from utils.train_utils import GradientMonitor, LossPlotterCallback

# Update config
PINN_CONFIG = {
    'input_dim': train_pts.shape[-1],
    'hidden_dim': hidden_dim,
    'num_layers': n_layers,
    'lr': lr,
    'epochs': max_epochs,
    'lambda_bc': lambda_bc,
    'use_fem_for_train': use_fem_for_train,
    'use_fem_for_test': use_fem_for_test,
    'n_collocation': len(train_pts), # Directly from the points we just created
    'n_boundary': n_bc
}

# my_physics = PoissonPhysics(source_type='sine', scale=1.0)
my_physics = PoissonGeneral(source_type='sine', scale=1.0)
pinn = PINNSystem(**PINN_CONFIG, physics=my_physics)

loss_plotter = LossPlotterCallback(
    model_name="Poisson PINN"
)

# Define Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=300, # Increased for PINNs as they can plateau
        mode='min', 
        min_delta=1e-6,
        verbose=True
    ),
    ModelCheckpoint(
        monitor='val_loss', 
        save_top_k=1, 
        mode='min',
        filename='best_pinn_{epoch}'
    ),
    GradientMonitor(verbose=True),
    loss_plotter
]

trainer = pl.Trainer(
    max_epochs=PINN_CONFIG['epochs'],
    accelerator="auto",
    devices=1,
    callbacks=callbacks,
    log_every_n_steps=20
)

from utils.reporting import print_config_summary

print_config_summary(pinn, PINN_CONFIG, model_type="PINN")

In [ ]:
import torch
print(f"¿Cuda disponible?: {torch.cuda.is_available()}")
print(f"¿Versión de CUDA de PyTorch?: {torch.version.cuda}")
print(f"¿Dispositivos detectados?: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Nombre de la GPU: {torch.cuda.get_device_name(0)}")

### 3.8. Execution, Inference and Comparison

Once the strategy is set and the model is assembled, we proceed to the final stage: training the "neural solver" and verifying if it has truly learned the laws of physics.

1. Training

    Before starting, we set torch.set_float32_matmul_precision('high'). This allows PyTorch to use the specialized Tensor Cores in modern GPUs (like NVIDIA Ampere or newer), significantly accelerating the training without compromising the precision required for PDEs.

    When we call trainer.fit(), the PINN begins its optimization loop. In each epoch, the network adjusts its internal weights to minimize the PDE Residual (internal consistency) and the Boundary Loss (sticking to the edges).

2. Inference
    After training, we switch the model to eval() mode. This is crucial because it deactivates specific training layers (like Dropout) and ensures the model is ready for pure prediction.
    - torch.no_grad(): We wrap the inference in this context to tell PyTorch not to calculate gradients, which saves memory and computing power.
    - The Goal: We pass the exact coordinates of the FEM mesh to the PINN. This allows us to have a point-by-point prediction that we can directly compare with the classical solution.

3. Visual comparison
    A scientific result is only valid if it can be quantified. We use two types of visualizations:
    - The Comparison Map: We plot the FEM solution side-by-side with the PINN prediction. At a glance, the physical patterns (the "shape" of the heat or pressure) should look identical.
    - The Error Map ($|u_{FEM} - u_{PINN}|$): This is the "Truth Map." It shows exactly where the neural network is struggling.

> **NOTE**: If you see high error concentrated only at the boundaries, it means your lambda_bc was too low. If the error is high everywhere, the network might need more layers (n_layers) or more training time.

In [ ]:
# 1. High precision
torch.set_float32_matmul_precision('high') 

# 2. Training loop
trainer.fit(pinn, train_dataloaders=train_loader, val_dataloaders=val_loader)

# 3. Inference
pinn.eval()
with torch.no_grad():
    x_test = torch.tensor(prob['doflocs'], dtype=torch.float32).to(pinn.device)
    u_pinn_pred = pinn(x_test).cpu().numpy().flatten()

# 4. Results comparison 
fig_comp = plot_comparison_with_pinn(pinn, prob['u'], prob['doflocs'])
plt.show()

# 5. Error analysis
fig_err = plot_error_analysis(u_fem=prob['u'], u_model=u_pinn_pred, model_name="PINN")
plt.show()


## 4. Graph Neural Network (GNN) Resolution

**What does it do?** A GNN operates on the mesh nodes. It uses **Message Passing** to gather information from neighbors and refine its prediction.
**Approach:** In this step, the GNN is **Supervised**. It learns to map node coordinates to the FEM solution $U(x, t)$.
**Connectivity:** Defined by the FEM mesh edges (adjacent neighbors).

### GNN Configuration

| Hyperparameter | Description | Value |
| :--- | :--- | :--- |
| `hidden_dim` | Width of Graph layers | 32 |
| `num_layers` | Message passing steps | 3 |
| `lr` | Learning rate | 1e-3 |
| `epochs` | Training iterations | 500 |
| `supervised` | Training against FEM targets | Yes |
| `connectivity` | Graph edges type | Mesh-based |

In [ ]:
dataset = PINNGraphDataset(nelem=nelem, porder=porder)
system_gnn = GNNSystem(hidden_dim=32, num_layers=3, lr=1e-3, supervised=True)
trainer_gnn = pl.Trainer(max_epochs=500, accelerator='auto', enable_checkpointing=False, log_every_n_steps=1)
trainer_gnn.fit(system_gnn, DataLoader(dataset, batch_size=1))

u_gnn = system_gnn(dataset[0]['x'].to(system_gnn.device), dataset[0]['edge_index'].to(system_gnn.device)).detach().cpu().numpy().flatten()
plot_comparison_with_fem(u_exact, u_gnn, doflocs, None, "GNN")
plt.show()

plot_error_analysis(u_exact, u_gnn, None, "GNN")
plt.show()

## 5. PINN vs GNN: Comparative Analysis

### Recap
- **PINN** learned the physics by looking at the interior PDE residuals. It is mesh-free but requires careful sampling.
- **GNN** learned the behavior of the solution by observing the FEM ground truth. It is extremely fast at inference and understands the domain topology.

| Feature | PINN | GNN (Supervised) |
| :--- | :--- | :--- |
| Need for Mesh | No | Yes |
| Learning Target | PDE Residual | FEM Data |
| Flexibility | High (any geometry) | Medium (restricted to graph) |
| Accuracy | Smooth/Global | Local/Topology-aware |

**Conclusion**: This tutorial demonstrates that both physics-informed and data-driven neural networks can approximate PDE solutions. In advanced cases, we combine these (PiGNN) to use the graph structure while enforcing physics residuals directly on the mesh nodes.